## 准备数据

In [48]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [49]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [50]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal(shape=[28*28, 100]))
        self.W2 = tf.Variable(tf.random.normal(shape=[100, 10]))
        self.b1 = tf.Variable(tf.ones(100))
        self.b2 = tf.Variable(tf.ones(10))

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        reshape_x = tf.reshape(x,[x.shape[0],x.shape[1]*x.shape[2]])

        # 隐藏层
        hiden_layer = tf.nn.relu(tf.matmul(reshape_x,self.W1)+self.b1)

        # 输出层
        logits = tf.matmul(hiden_layer,self.W2)+self.b2

        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [51]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [53]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 30.42042 ; accuracy 0.31065
epoch 1 : loss 30.054417 ; accuracy 0.31553334
epoch 2 : loss 29.69847 ; accuracy 0.31991667
epoch 3 : loss 29.352203 ; accuracy 0.32418334
epoch 4 : loss 29.01535 ; accuracy 0.3288
epoch 5 : loss 28.687504 ; accuracy 0.33308333
epoch 6 : loss 28.368439 ; accuracy 0.3375
epoch 7 : loss 28.057867 ; accuracy 0.34148332
epoch 8 : loss 27.755424 ; accuracy 0.34583333
epoch 9 : loss 27.460817 ; accuracy 0.35041666
epoch 10 : loss 27.173841 ; accuracy 0.3551
epoch 11 : loss 26.894245 ; accuracy 0.35936666
epoch 12 : loss 26.621801 ; accuracy 0.36285
epoch 13 : loss 26.35615 ; accuracy 0.3668
epoch 14 : loss 26.09692 ; accuracy 0.37066665
epoch 15 : loss 25.843863 ; accuracy 0.37453333
epoch 16 : loss 25.596708 ; accuracy 0.37816668
epoch 17 : loss 25.355165 ; accuracy 0.38173333
epoch 18 : loss 25.119013 ; accuracy 0.38508335
epoch 19 : loss 24.8881 ; accuracy 0.38833332
epoch 20 : loss 24.662315 ; accuracy 0.39185
epoch 21 : loss 24.441483 ; accura